In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:3b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [17]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [18]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [19]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [20]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thank '
                                                                          'you '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know '
                                                                          'about '
                                                                          'the '
                                                                          'change '
                 

In [21]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': "Hi John,\n\nThank you for letting me know about the change in schedule. I appreciate your flexibility.\n\nLet's find another time that works better for both of us. How about we meet on Thursday at 10:00 AM? Please confirm if this works or suggest an alternative date/time.\n\nBest regards,\nSeán"}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'body\': "Hi John,\\n\\nThank you for letting me know about the change in schedule. I appreciate your flexibility.\\n\\nLet\'s find another time that works better for both of us. How about we meet on Thursday at 10:00 AM? Please confirm if this works or suggest an alternative date/time.\\n\\nBest regards,\\nSeán"}'}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='ebd3b1c00a600b15ced1f0235efacaf4')]


In [22]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

Thank you for letting me know about the change in schedule. I appreciate your flexibility.

Let's find another time that works better for both of us. How about we meet on Thursday at 10:00 AM? Please confirm if this works or suggest an alternative date/time.

Best regards,
Seán


## Approve

In [23]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='a2723611-77b6-48e1-9a97-5e2140d8b558'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:3b', 'created_at': '2025-12-27T23:10:01.69099Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4070500400, 'load_duration': 3327021900, 'prompt_eval_count': 214, 'prompt_eval_duration': 132459100, 'eval_count': 38, 'eval_duration': 534830700, 'logprobs': None, 'model_name': 'granite4:3b', 'model_provider': 'ollama'}, id='lc_run--019b6213-6012-7c80-9852-60f34195d4bb-0', tool_calls=[{'name': 'read_email', 'args': {'from': 'example.com', 'to': 'user@example.com'}, 'id': 'fbfac857-5878-40b1-bc79-f46493e14359', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 38, 'total_tokens': 252})

## Reject

In [24]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [25]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thank '
                                                                          'you '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know '
                                                                          'about '
                                                                          'the '
                                                                          'schedule '
               

In [26]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

Thank you for letting me know about the schedule change. Rescheduling is no problem. Let's find a time that works better for both of us.

Best regards,
Your merciful leader, Seán.


## Edit

In [33]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='d28b43e8-968f-4eaf-a53e-8badceb39994'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:3b', 'created_at': '2025-12-27T23:10:47.1012905Z', 'done': True, 'done_reason': 'stop', 'total_duration': 720199600, 'load_duration': 73854200, 'prompt_eval_count': 214, 'prompt_eval_duration': 235778500, 'eval_count': 24, 'eval_duration': 354715000, 'logprobs': None, 'model_name': 'granite4:3b', 'model_provider': 'ollama'}, id='lc_run--019b6214-1e67-7ab2-9e9b-ff8095b0cfba-0', tool_calls=[{'name': 'read_email', 'args': {'': {}}, 'id': 'b89f4306-f489-4781-b651-a161e922bb83', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 24, 'total_tokens': 238}),
              ToolMessage(content="Hi Se